In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from loguru import logger

# 1. Linear data
## 1.1 generating synthetic data
Let's create some linear data

In [ ]:
def make_linear(
    filename: Path, size: int = 100, a: float = 2, b: float = 4, s: float = 0.5
):
    if filename.exists():
        logger.info(f"Data already generated in {filename.absolute()}")
    else:
        x = np.linspace(0, 1, size)
        noise = np.random.normal(scale=s, size=len(x))
        y = a * x + b + noise
        pd.DataFrame({"x": x, "y": y}).to_csv(filename, index=False)
        logger.info(f"Data generated at {filename.absolute()}.")
    return filename

In [ ]:
datadir = Path("../data/sim")
if not datadir.exists():
    datadir.mkdir(parents=True)

In [ ]:
var_a = 2
var_b = 4
filename = datadir / f"linear_a{var_a}_b{var_b}.csv"
make_linear(filename, a=var_a, b=var_b)
df = pd.read_csv(filename)
plt.plot(df["x"], df["y"], "b+")

The formula for a line is:

$$model(x)= a * x + b$$

We have set the slope $a=2$ and the intercept $b=4$.

## 1.2 Statistical modelling

It's obvisous that there is a lot of noise, but let's try to fit a linear model.

We will use pymc for the sampling.

In [ ]:
import pymc as pm

Now, we will assume that both $a$ and $b$ follow a normal distribution.
We can write this down with this notation:

$$a \sim \mathcal{N}(\mu, \sigma)$$
$$b \sim \mathcal{N}(\mu, \sigma)$$

This means: we model $a$ and $b$ as normally distributed with mean $\mu$ and standard deviation $\sigma$, even if we dont know (yet) what those exact parameters are.

In [ ]:
df = pd.read_csv(filename)
x = df["x"]
y = df["y"]

model = pm.Model()
with model:
    a = pm.Normal("a")
    b = pm.Normal("b")

As you can see below, the model sort of what we expected. You can also see that we start with $\mu=0$ and $\sigma=1$, these are just starting points (we have to start somewhere) and this will converge when we look at data. If you are wondering "yeah but what if I have some vague idea about the mean value and want to model that too" you are on the right track! For now, let's keep it simple.

In [ ]:
model



We will assume that our model is not perfect. There will be noise.:

$$model(x) = a * x + b + noise$$

But we can assume the noise itself will also follow a normal distribution, with a mean $\mu$ and a standard deviation $\sigma$:
$$ noise \sim \mathscr{N}(\mu, \sigma)$$

This means there are 4 parameters we want to figure out:
 - slope $a$
 - intercept $b$
 - mean $\mu$ from the noise
 - standard deviation $\sigma$ from the noise

We should be able to calculate the $\mu$, because we expect:
 $$model(x) - (a*x + b) = \mathscr{N}(0, \sigma)$$

 which is the same as:

 $$model(x) = \mathscr{N}(a*x+b, \sigma)$$
or, more general,
 $$model(x) = \mathscr{N}(f_{\theta}(x), \sigma)$$

Where $f$ is some function, and $\theta$ are the learnable parameters of the model. In this case, $\theta = (a, b)$.

At first, this might seem confusing. Why is the mean suddenly a function? Shouldn't the mean be a fixed number? 

What we are actually saying is this: lets imagine you want to sample our model for a given x, eg `x=0.4`. Because our model is probabilistic, we don't expect to get the same values every time. The results are a random process. If you would sample the model with `x=0.4` a 100 times, we assume output would follow a distribution. What sort of distribution? Well, in this model, a normal distribution.


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
noise = y.values - (x.values * var_a + var_b)
ax[0].plot(x.values, y.values, "b+")
ax[0].set_title("Values with linear trend")
ax[1].plot(x.values, noise, "b+")
ax[1].set_title("Residual noise: Values with linear subtracted")


If I take the observed value $y$ and subtract the model $ax + b$, I keep the noise and that noise will have a mean of 0 and an unknown standard deviation of $\sigma$. If I dont subtract the model, I no longer have a mean of zero, but a mean of $ax + b$. 

 ## 1.3 How to model the standard deviation?
 Because a standard deviation can't be negative, we could run into problems if we model it as a normal distribution. We run the chance of sampling negative values, which will cause errors. So, we will need a distribution that is strictly positive.

 We can use a halfnormal distribution for this, which is just a normal distribution, but one that cant become negative. There are better options for modeling standard deviations, but let's start with this for now.

 the `with` syntax allows us to extend our existing model. We can add new variables to the model, and we can also add new observations to the model. This is a very powerful feature of pymc3, and it allows us to build complex models in a very readable way.

In [ ]:
with model:
    sigma = pm.HalfNormal("sigma")
    predict = a * x + b
    estimate = pm.Normal("y", mu=predict, sigma=sigma, observed=y)

model

As you can see, we expanded our model. We now have some sort of "causal chain" of variables.
We start with $a$ and $b$ and $sigma$, and those are combined into our model $(a*x + b)$, and the output of the this prediction is taken to be the mean of the actual value plus noise.

## 1.4 Sampling
With this model, we can run simulations to figure out the best fit.

In [ ]:
with model:
    result = pm.sample(3000)

Note pm uses 4 chains. This is similar to what with did with estimating $\pi$: every chain has its own randomness, but combining 4 chains and taking the average of every chain reduces the randomness.

In [ ]:
import arviz as az

az.plot_trace(result)
plt.tight_layout()

That's a really nice answer!

Because we generated the data for ourselves, we actually know that the true answers are $a=2.0$, $b=4.0$ and $\sigma=0.5$.

All the answers are pretty close! And we also have a good grip on how certain we are, given the limited amount of data. 

Using more data would decrease our uncertainty.

# 2. Modeling more complex functions: Sinewave

## 2.1 Generating synthetic data

Changing the formula of the model follows the same template. I will demonstrate the same pattern, but this time with a sine wave.

Let's make a sine wave with an amplitude of 3, frequency of 2, and some noise.

In [ ]:
def make_sine_wave(
    filename: Path,
    a: float,
    f: float,
    s: float,
    t: np.ndarray = np.linspace(0, 4, 100),
) -> None:
    if filename.exists():
        logger.info(f"Data already generated in {filename.absolute()}")
    else:
        noise = np.random.normal(scale=s, size=len(t))
        v = a * np.sin(f * t) + noise
        pd.DataFrame({"time": t, "value": v}).to_csv(filename, index=False)
        logger.info(f"Data generated at {filename.absolute()}.")
    return filename

In [ ]:
var_a = 3
var_f = 2
var_s = 0.2
filename = datadir / f"sine_a{var_a}_f{var_f}_s{var_s}.csv"
make_sine_wave(filename, a=var_a, f=var_f, s=var_s)

In [ ]:
df = pd.read_csv(filename)
plt.plot(df["time"], df["value"], "b+")
plt.xlabel("time")
plt.ylabel("value");

## 2.2 Weakly informative distributions
As you can see, there is some noise, but not too much.
We got good results with the halfnorm distribution, but we can do better.

As a modeller, it is our task to express our knowledge about the situation in assumptions. We try to stay on the safe side, but adding a little bit of knowledge to our model is a good idea.

Because, saying we know absolutely nothing more about the standard deviation than that it is positive is a bit too strong.

We could speed up the calculations by using [what is called a "weakly informative" distribution.](http://www.stat.columbia.edu/~gelman/research/published/taumain.pdf)

Basically, we want to add just a little bit more of knowledge to our beliefs about the standard deviation, but not too much.

The prior should be picked, such that the information it does provide is intentionally weaker than whatever actual knowledge is available, just to be on the safe side.

For example, we can be sure the standard deviaition is below 100. Also, it expect it to be somewhat above 0.

The paper linked suggests using an inverse gamma.
Lets plot the different options, comparing some distributions, to find out if we can better understand why that makes sense.

In [ ]:
from scipy import stats

fig, ax = plt.subplots(1, 3, figsize=(12, 4))

x = df["time"]
ax[0].plot(x, stats.halfnorm.pdf(x))
ax[0].set_title("Halfnorm")

ax[1].plot(x, stats.halfcauchy.pdf(x))
ax[1].set_title("Half Cauchy")

ax[2].plot(x, stats.invgamma.pdf(x, a=1))
ax[2].set_title("Inverse Gamma")

To me, the inverse gamma indeed looks like what I would expect. Not zero, very high is also unlikely, and somewere between 0.1 and 3 seems very reasonable as a guess.

If you are afraid of picking the wrong distribution, no worries! As you have seen before, a Halfnormal will work just fine. Even starting with a uniform distribution will work, even though it will probably be less efficient.
The Half Cauchy will also work fine in this case, as will the Inverse Gamma. However, if your data becomes more complex, picking the right balance between more neutral distributions (eg uniform) and more informed distributions, like the Inverse Gamma, could make a difference.

In general, because it is reasonable to assume that the standard deviation will neither be 0, nor very high, the inverse gamma will converge faster in most cases.

## 2.3 Sine wave model

In this case, our model looks like this:

$$Amp \sim \mathcal{N}(0, 1)$$
$$freq \sim \mathcal{N}(0, 1)$$
$$predict \sim Amp * sin(freq * x)$$
$$\sigma \sim InverseGamma(a=1)$$
$$y \sim \mathcal{N}(predict, \sigma)$$


If this is confusing, try to read it from the bottom up: we model the residual as a normal distribution. The residual will have our predcitive model as the mean, and there will be some uncertainty in our model that is described by the standard deviation. Higher up in the "causal chain" we find the standard deviation $\sigma$, which we are uncertain about and model with an InverseGamma, and also parameters of the model that we want to find out (like the amplitude and the frequency) and model with a normal distribution.

Sometimes people draw diagrams with arrows to make the causal route in their probabilistic models more explicit.

## 2.4 Sampling
Now lets implement the model:


In [ ]:
import pymc as pm

df = pd.read_csv(filename)
x = df["time"]
y = df["value"]

model = pm.Model()

with model:
    Amplitude = pm.Normal("amp")
    freq = pm.Normal("f")
    predict = Amplitude * np.sin(freq * x)

    sigma = pm.InverseGamma("sigma", alpha=1)
    estimate = pm.Normal("y", mu=predict, sigma=sigma, observed=y)

    result = pm.sample(3000)
az.plot_trace(result)
plt.tight_layout()

## 2.5 Improving the model
Sometimes the amplitude and frequency have both negative and positive values. This is caused by the way the sine wave formula works. 
Both actually give the same result, but the sampler might inform you that it wasn't able to converge properly. 

If you dont like that, you could force the amplitude and frequency to be positive, for example with a HalfNormal distribution (that only allows positive values), or you could also use a uniform distribution with a lower bound of 0.

In [ ]:
model = pm.Model()

with model:
    Amplitude = pm.HalfNormal("amp")
    freq = pm.HalfNormal("f")
    predict = Amplitude * np.sin(freq * x)

    sigma = pm.InverseGamma("sigma", alpha=1)
    estimate = pm.Normal("y", mu=predict, sigma=sigma, observed=y)

    result = pm.sample(3000)
az.plot_trace(result)
plt.tight_layout()

## 2.6 Plotting the results
We can plot the final result, adding two standard deviations as upper and lower bounds:

In [ ]:
A = result.posterior.amp.median().values
f = result.posterior.f.median().values
sigma = result.posterior.sigma.median().values
yhat = A * np.sin(f * df["time"])

plt.figure(figsize=(12, 6))
plt.plot(df["time"], df["value"], "b+")
plt.plot(df["time"], yhat, color="black", label="model")
plt.fill_between(df["time"], yhat + sigma, yhat - sigma, color="r", alpha=0.5)
plt.fill_between(df["time"], yhat + 2 * sigma, yhat - 2 * sigma, color="r", alpha=0.3);